<a href="https://colab.research.google.com/github/Richie7x/Image-Classification-Model/blob/main/Minor_Project_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing autotime to show execution time
!pip install ipython-autotime
%load_ext autotime

- COLLECTING DATA

In [ ]:
#Installing webscraping tool to get images
!pip install bing-image-downloader 

#Creating a directory for saving images
!mkdir Images 

#Webscraping images from bing
from bing_image_downloader import downloader
downloader.download("drivers license",limit=30,output_dir='Images',adult_filter_off=True)
downloader.download("U.S. Passport cover",limit=30,output_dir='Images',adult_filter_off=True)
downloader.download("Print Blank DD214 Form",limit=30,output_dir='Images',adult_filter_off=True)

- PREPROCESSING THE DATA

In [ ]:
import os #-------------------------------------To work around folders
import matplotlib.pyplot as plt #---------------To plot pixels as a whole image
import numpy as np     
from skimage.io import imread #-----------------To read images 
from skimage.transform import resize #----------To resize the images 

goal = []
images = []
flatimg = []

dir = '/content/Images'
cats = ['drivers license','U.S. Passport cover','Print Blank DD214 Form']

for cat in cats:
  code = cats.index(cat) #----------------------Label Encoding 
  path = os.path.join(dir,cat) #----------------Creating Path
  for img in os.listdir(path):
    picture = imread(os.path.join(path,img))
    resizedimg = resize(picture,(150,150,3)) #--Resizing, also normalizes the value between 0 to 1
    flatimg.append(resizedimg.flatten()) #------Flattening the Image
    images.append(resizedimg)
    goal.append(code)

#Converting Lists into numpy arrays
flatimg = np.array(flatimg)
goal = np.array(goal)
images = np.array(images)


- APPLYING ALGORITHM

In [ ]:
#Splitting data into training and testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flatimg,goal,test_size=0.3,random_state=109)

from sklearn.model_selection import GridSearchCV
from sklearn import svm
parameters = [
              {'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']}
]

svc = svm.SVC(probability=True)
model = GridSearchCV(svc,parameters)
model.fit(x_train,y_train)

#Predicted Output
y_pred = model.predict(x_test)
print("Predicted Output =", y_pred)

#Test Output
print("Test Output =", y_test)


#Testing the accuracy of the model
from sklearn.metrics import accuracy_score,confusion_matrix
print("Model Accuracy = " + str(accuracy_score(y_pred,y_test)*100)+"%")

#Saving the model using pickle library
import pickle
pickle.dump(model,open('img_model.p','wb'))

Predicted Output = [1 0 1 2 2 1 0 1 1 0 1 1 0 2 2 1 2 1 0 1 1 2 2 0 0 0 1]
Test Output = [1 0 1 2 2 1 0 1 1 0 1 0 1 2 2 1 2 1 0 1 2 2 2 0 0 0 1]
Model Accuracy = 88.88888888888889%
time: 1min 16s (started: 2021-06-19 06:40:17 +00:00)


- TESTING THE MODEL

In [ ]:
flatimg = []
url = input("Enter image URL - ")
img = imread(url)
resizedimg = resize(img,(150,150,3))
flatimg.append(resizedimg.flatten())
flatimg = np.array(flatimg)
plt.imshow(resizedimg)
y_out = model.predict(flatimg)

if y_out == 0:
  y_out = "Driver's License"
elif y_out == 1:
  y_out = "Passport"
elif y_out == 2:
  y_out = "DD214 (Military Form)"
else:
  y_out = "Other Image"

print("\n\nPREDICTED OUTPUT : " + y_out +"\n")



In [ ]:
#Match percentage of tested data with respect to the given categories 
x = model.predict_proba(flatimg)
for index,item in enumerate(cats):
  print(str(item) + " = " + str(round((x[0][index]*100),2)) + "%")